In [1]:
import "fmt"

### Method declaration and call

#### Method on a simple type

In [2]:


type Percent int

func (p Percent) toProbabilty() int{
    return int(p)/100 
}

all := Percent(100)

fmt.Println(all.toProbabilty())

1


2 <nil>

#### Method on a slice

In [3]:
import "math"

type Vector []float64

func (v Vector) Length() float64{
    sumSquare := 0.0
    for _,c := range v{
        sumSquare+= c*c
    }
    return math.Sqrt(sumSquare)
}

v := Vector{3.0,4.0}

fmt.Println(v.Length())

5


2 <nil>

#### Method on a struct

In [4]:
type Point struct{X,Y float64}

func (p Point) toVector() Vector{
    return Vector{p.X,p.Y}
}

fmt.Println(Point{6,8}.toVector().Length())

10


3 <nil>

In [5]:
func (p Point) getX(scale float64) float64{
    return p.X*scale
}
func (p Point) getY(scale float64) float64{
    return  p.Y*scale
}


// Note the function type
var pointFun func(Point,float64)float64 = Point.getX

pointFun(Point{2,3},3)

6

#### Method on a function

In [6]:
type Condition func(number int)bool

func (c Condition) AssertTrue(number int)bool{
    return c(number)
}

func (c Condition) AssertFalse(number int)bool{
    return !c(number)
}

validAge := Condition(func(age int)bool{
    return age > 18
})

fmt.Printf("19 is valid age? %v\n",validAge.AssertTrue(19))
fmt.Printf("17 is in-valid age? %v\n",validAge.AssertFalse(17))

19 is valid age? true
17 is in-valid age? true


25 <nil>

### Methods with pointer receiver

In [7]:
func (p *Point) Scale(factor float64){
    p.X *= factor
    p.Y *= factor
}

point := Point{3,4}

pointer := &point

pointer.Scale(2)

fmt.Println(point)

(&point).Scale(2)

fmt.Println(point)

// OR very simple: here compiler does &point conversion implicitly

point.Scale(3)

fmt.Println(point)

// Point{4,5}.scale(7) // This doesn't work because we cannot take address of Point{4,5}

{6 8}
{12 16}
{36 48}


8 <nil>

In [8]:
func (p Point) Slope() float64{
    return float64(p.Y)/float64(p.X)
}

fmt.Println(Point{4,8}.Slope())


pointer = &Point{3,4}


// Compiler does *pointer conversion implicitly
fmt.Println(pointer.Slope())

fmt.Println((&Point{5,6}).Slope())


2
1.3333333333333333
1.2


4 <nil>

#### Nil as receiver

In [9]:
type Segment struct{
    start,end Point
}

func (s *Segment) Slope() float64{
    if(s==nil){
        return math.NaN()
    }
    return float64(s.end.Y - s.start.Y)/float64(s.end.X - s.start.X)
}

var s *Segment = nil

fmt.Println(s.Slope())

NaN


4 <nil>

### Composition with struct

In [10]:
type Color struct{
    R,G,B int
    
}

func (c Color) Sum() int{
    return c.R+c.G+c.B
}

type Shape struct{
    noOfSides int
    Color
}

func (s Shape) String() string{
    return fmt.Sprintf("Shape with noOfSides %d ",s.noOfSides)
}

type Triangle struct{
    Shape
}

func (t Triangle) String() string{
    return fmt.Sprintf("Triangle")
}

triangle := Triangle{{3,{255,0,0}}}

fmt.Println(triangle.Sum()) // Sum promoted from Color

fmt.Println(triangle.String()) // Shape#String is hidden by Triangle#String

255
Triangle


9 <nil>

### Method values

These are useful when we need to keep the receiver fixed and embedded in the function

In [11]:
func (this Point) Distance (another Point) float64{
    return math.Sqrt(math.Pow(float64(this.X-another.X),2)+math.Pow(float64(this.Y-another.Y),2))
}


In [19]:
theCenter := Point{0,0}

var distanceFromCenter func(Point)float64 = theCenter.Distance // This is method value, the center point it part of the method itself,
// and no more a parameter


fmt.Println(distanceFromCenter(Point{3,4}))

fmt.Println(distanceFromCenter(Point{8,6}))

5
10


3 <nil>

### Method expressions

convert methods to functions where first argument is the receiver type

In [17]:
var distanceCalc func(Point,Point)float64  = Point.Distance

fmt.Println(distanceCalc(Point{1,2},Point{4,6})) // note that the first argument was reciever type in the method

5


2 <nil>

In [29]:
type Dog string

func (dog *Dog) sound(){
    fmt.Println("Bowbow")
}

// Following is incorrect

/*sound := Dog.sound
dog := Dog("bruno")
sound(dog)*/

// Follows the correct way to collect method expression 

sound := (*Dog).sound // first argument is a pointer (to receiver) in the sound method
dog := Dog("bruno")
sound(&dog) // address of dog because the argument is a pointer


ERROR: repl.go:15:12: type <*main.Dog> has no method "sound": (*Dog).sound